In [9]:
import sys
sys.path.insert(1, 'BATTERY_GITHUB')
import eval_utils
import pre_process_utils
# import BATTERY_GITHUB.pre_process_utils as pre_process_utils
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import gridspec
params = {'legend.fontsize': 20,
          'figure.figsize': (9,6),
         'axes.labelsize': 20,
         'axes.titlesize':20,
         'xtick.labelsize':'xx-large',
         'axes.linewidth' : 2,
         'ytick.labelsize':'xx-large'}

plt.rcParams.update(params)

In [25]:
import importlib
importlib.reload(eval_utils)

<module 'eval_utils' from '/baja/BATTERY_GITHUB/eval_utils.py'>

In [31]:
# HI DATA
data = np.load('Data/Battery/Load_data_big_sameT/processed_data_2post_200_60RUL.npz')
Voltage_windows = data['Voltage_windows']
Cycles_windows = data['Cycles_windows']
Units_windows = data['Units_windows']
Current_windows = data['Current_windows']
Cycles_windows_2 = data['Cycles_windows_2']
HI_windows = data['HI_windows']
Temp_windows = data['Temp_windows']
Time_windows = data['Time_windows']



data = np.load('Data/Battery/Load_data_big_sameT/processed_data_2post_test_200_60RUL.npz')
Voltage_windows_test = data['Voltage_windows_test']
Cycles_windows_test = data['Cycles_windows_test']
Units_windows_test = data['Units_windows_test']
Current_windows_test = data['Current_windows_test']
Cycles_windows_2_test = data['Cycles_windows_2_test']
HI_windows_test = data['HI_windows_test']   
Temp_windows_test = data['Temp_windows_test']
Time_windows_test = data['Time_windows_test']

In [32]:
Y_windows = pre_process_utils.RUL_maker(Cycles_windows,Units_windows).reshape(-1,1)
Y_windows_test = pre_process_utils.RUL_maker(Cycles_windows_test,Units_windows_test).reshape(-1,1)

#Convert to SOH%
MANUFACTURE_CAPACITY = 2.1
HI_windows = HI_windows/MANUFACTURE_CAPACITY
HI_windows_test = HI_windows_test/MANUFACTURE_CAPACITY
V_T = np.concatenate((Voltage_windows, Temp_windows), axis=-1)
V_T_test = np.concatenate((Voltage_windows_test,Temp_windows_test),axis = -1)

In [33]:
HI_windows.shape

(418317, 200, 1)

# BASELINE RUL

In [13]:
def RUL_model(t=64,
              feature_X_in=2,
              feature_W_in=1,
              feature_H_in=1,
              feature_out_size=1,
              activation='relu',
              filter=[10,10,1],
              filter_size=10,
              useH=True):
    '''
    RUL_model: Generates a model for predicting Remaining Useful Life (RUL).

    Args:
        t (int): Time steps.
        feature_X_in (int): Number of features in input X.
        feature_W_in (int): Number of features in input W.
        feature_H_in (int): Number of features in input H.
        feature_out_size (int): Number of features in the output.
        activation (str): Activation function to use in the hidden layers.
        filter (list): List containing the number of filters for each convolutional layer.
        filter_size (int): Size of the convolutional filters.
        useH (bool): Flag indicating whether to include H as input.
                     If True, input will be [X, W, H, T].
                     If False, input will be [X, W, T].

    Returns:
        tf.keras.models.Model: RUL prediction model.

    Note:
        If useH is True, the model will accept input [X, W, H, T] and output Y.
        If useH is False, the model will accept input [X, W, T] and output Y.
    '''

    # Define input layers for X, W, T, and H
    x_in = tf.keras.layers.Input(shape=(t, feature_X_in), name="X_in")
    w_in = tf.keras.layers.Input(shape=(t, feature_W_in), name="W_in")
    t_in = tf.keras.layers.Input(shape=(t, 1), name="T_in")
    
    # Concatenate input data based on the useH flag
    if useH:
        h_in = tf.keras.layers.Input(shape=(t, feature_H_in), name="H_in")
        x = tf.keras.layers.Concatenate(axis=-1)([x_in, w_in, h_in, t_in])
    else:
        x = tf.keras.layers.Concatenate(axis=-1)([x_in, w_in, t_in])
      
    # Apply convolutional layers
    for i in filter:
        x = tf.keras.layers.Conv1D(i, filter_size, 1, padding='same', activation=activation)(x)

    # Flatten the output
    x = tf.keras.layers.Flatten()(x)
    
    # Apply dense layers
    y = tf.keras.layers.Dense(50, activation=activation)(x)
    y = tf.keras.layers.Dense(feature_out_size, activation='linear')(y)

    # Create the model with the appropriate inputs and outputs
    if useH:
        model = tf.keras.models.Model([x_in, w_in, t_in, h_in], y)
    else:
        model = tf.keras.models.Model([x_in, w_in, t_in], y)

    return model


In [41]:
useH=False
layers=  [10,10,10]
Window_size = Voltage_windows.shape[1]
model = RUL_model(Window_size,filter = layers, useH = useH)

In [42]:
W = Voltage_windows.shape[1]
cycle_scaler = MinMaxScaler()
c_temp = cycle_scaler.fit_transform(Cycles_windows.reshape(-1,1))
c_temp = np.repeat(c_temp,W,1).reshape(-1,W,1)
c_temp_test = cycle_scaler.transform(Cycles_windows_test.reshape(-1,1))
c_temp_test = np.repeat(c_temp_test,W,1).reshape(-1,W,1)


In [43]:
result = eval_utils.train_and_evaluate_rul_model(V_T, Current_windows,c_temp, Cycles_windows, Units_windows, Y_windows,HI_windows,
                                                     V_T_test, Current_windows_test, c_temp_test,Cycles_windows_test, Units_windows_test, Y_windows_test,HI_windows_test,
                                             model,useH=False,runs = 1, epochs = 30, batch_size=2048,learning_rate = 0.001,reset_weights = True)

RUN#:  0
reset_weights
Epoch 1/30
184/184 [==============================] - 6s 18ms/step - loss: 171.7829 - val_loss: 112.2604
Epoch 2/30
184/184 [==============================] - 4s 19ms/step - loss: 111.5675 - val_loss: 111.4915
Epoch 3/30
184/184 [==============================] - 4s 18ms/step - loss: 110.8543 - val_loss: 111.0446
Epoch 4/30
184/184 [==============================] - 4s 17ms/step - loss: 110.4813 - val_loss: 110.7173
Epoch 5/30
184/184 [==============================] - 4s 17ms/step - loss: 110.2375 - val_loss: 110.4514
Epoch 6/30
184/184 [==============================] - 4s 17ms/step - loss: 110.1227 - val_loss: 110.5412
Epoch 7/30
184/184 [==============================] - 4s 18ms/step - loss: 109.9756 - val_loss: 110.2084
Epoch 8/30
184/184 [==============================] - 4s 17ms/step - loss: 109.8516 - val_loss: 110.1528
Epoch 9/30
184/184 [==============================] - 4s 17ms/step - loss: 109.7430 - val_loss: 109.9534
Epoch 10/30
184/184 [===========

In [44]:
pd.DataFrame(result)

,MAE_TR,MAPE_TR,RMSE_TR,MAE_TS,MAPE_TS,RMSE_TS,TIME,I
0,197.413431,244.339192,239.134334,146.703545,226.258451,170.808691,136.625927,0


# GT RUL

In [28]:
useH=True
layers=  [10,10,10]
Window_size = Voltage_windows.shape[1]
model = RUL_model(Window_size,filter = layers, useH = useH)

In [29]:
W = Voltage_windows.shape[1]
cycle_scaler = MinMaxScaler()
c_temp = cycle_scaler.fit_transform(Cycles_windows.reshape(-1,1))
c_temp = np.repeat(c_temp,W,1).reshape(-1,W,1)
c_temp_test = cycle_scaler.transform(Cycles_windows_test.reshape(-1,1))
c_temp_test = np.repeat(c_temp_test,W,1).reshape(-1,W,1)


In [34]:
result = eval_utils.train_and_evaluate_rul_model(V_T, Current_windows,c_temp, Cycles_windows, Units_windows, Y_windows,HI_windows,
                                                     V_T_test, Current_windows_test, c_temp_test,Cycles_windows_test, Units_windows_test, Y_windows_test,HI_windows_test,
                                             model,useH=True,runs = 1, epochs = 30, batch_size=2048,learning_rate = 0.001,reset_weights = False)

RUN#:  0
reset_weights
Epoch 1/30
184/184 [==============================] - 8s 27ms/step - loss: 161.2007 - val_loss: 104.2104
Epoch 2/30
184/184 [==============================] - 5s 25ms/step - loss: 98.4484 - val_loss: 88.2758
Epoch 3/30
184/184 [==============================] - 5s 24ms/step - loss: 78.7570 - val_loss: 72.8866
Epoch 4/30
184/184 [==============================] - 5s 23ms/step - loss: 67.9221 - val_loss: 64.9281
Epoch 5/30
184/184 [==============================] - 5s 24ms/step - loss: 64.4527 - val_loss: 62.6063
Epoch 6/30
184/184 [==============================] - 5s 23ms/step - loss: 64.1789 - val_loss: 67.3980
Epoch 7/30
184/184 [==============================] - 5s 23ms/step - loss: 62.1697 - val_loss: 63.1665
Epoch 8/30
184/184 [==============================] - 5s 23ms/step - loss: 61.3746 - val_loss: 62.7764
Epoch 9/30
184/184 [==============================] - 5s 23ms/step - loss: 60.4745 - val_loss: 58.7053
Epoch 10/30
184/184 [===========================

In [35]:
pd.DataFrame(result)

,MAE_TR,MAPE_TR,RMSE_TR,MAE_TS,MAPE_TS,RMSE_TS,TIME,I
0,207.12815,290.031666,253.683062,104.925207,73.759906,125.791054,168.257442,0
